# Kyoto gas splitter
This notebook uses the kyoto gas splitter to divide up the greenhouse gas projections from the CLimate Action Tracker project. 

In [1]:
import os.path

import pandas as pd
import pyam
import matplotlib.pyplot as plt
import numpy as np
from silicone.utils import (
    _get_unit_of_variable,
    find_matching_scenarios,
    download_or_load_sr15,
    convert_units_to_MtCO2_equiv
)
import silicone.multiple_infillers as mi

<IPython.core.display.Javascript object>

pyam - INFO: Running in a notebook, setting `pyam` logging level to `logging.INFO` and adding stderr handler


In [2]:
input_kyoto_file = "./../input/DownloadedClimateActionTrackerData.csv"
input_complete_data = "./../input/sr15_database.csv"
df = download_or_load_sr15(input_complete_data)

pyam.utils - INFO: Reading `./../input/sr15_database.csv`


We have to format the CVS file to make a pyam dataframe.

In [3]:
kyoto_csv = pd.read_csv(input_kyoto_file, header=[0, 1], index_col=0)
kyoto_csv = kyoto_csv.transpose().reset_index()
kyoto_csv.head()

,level_0,level_1,2010,2011,2012,2013,2014,2015,2016,2017,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
0,AR5 BAU,high,49,51,52,54,56,58,59,61,...,163,164,165,167,168,170,171,172,174,175
1,AR5 BAU,low,44,45,46,47,48,49,50,51,...,87,87,86,86,85,85,84,84,83,83
2,Pledges and targets,high,48,49,50,50,50,50,50,51,...,36,36,35,35,35,34,34,34,34,33
3,Pledges and targets,low,48,49,50,50,50,50,50,51,...,28,28,28,27,27,27,27,26,26,26
4,Current policy projections,high,48,49,50,50,50,51,51,51,...,55,55,55,55,54,54,54,54,54,54


In [4]:
kyoto_csv = kyoto_csv.rename(columns={"level_0": "model", "level_1": "scenario"})

In [11]:
aggregate = "Emissions|Kyoto Gases (AR4-GWP100)"
kyoto_csv["variable"] = aggregate
kyoto_csv["unit"] = "Mt CO2-equiv/yr"
kyoto_csv["region"] = "World"

In [12]:
kyoto_df = pyam.IamDataFrame(kyoto_csv)
kyoto_df.head()

,model,scenario,region,variable,unit,year,value
0,1.5C consistent,high,World,Emissions|Kyoto Gases (AR4-GWP100),Mt CO2-equiv/yr,2010,48.0
1,1.5C consistent,high,World,Emissions|Kyoto Gases (AR4-GWP100),Mt CO2-equiv/yr,2011,49.0
2,1.5C consistent,high,World,Emissions|Kyoto Gases (AR4-GWP100),Mt CO2-equiv/yr,2012,49.0
3,1.5C consistent,high,World,Emissions|Kyoto Gases (AR4-GWP100),Mt CO2-equiv/yr,2013,50.0
4,1.5C consistent,high,World,Emissions|Kyoto Gases (AR4-GWP100),Mt CO2-equiv/yr,2014,50.0


Now we can set up the variables to decompose the aggregate into its components. 

In [7]:
components = [
    "Emissions|CO2",
    "Emissions|CH4",
    "Emissions|N2O",
    "Emissions|F-Gases",
]
unit_consistent_db = convert_units_to_MtCO2_equiv(
    df.filter(variable=components)
)
unit_consistent_db.variables(True)

,variable,unit
0,Emissions|CH4,Mt CO2-equiv/yr
1,Emissions|CO2,Mt CO2/yr
2,Emissions|F-Gases,Mt CO2-equiv/yr
3,Emissions|N2O,Mt CO2-equiv/yr


Our complete dataframe has two extra columns and needs to be made consistent with the kyoto gas dataframe. 

In [8]:
del unit_consistent_db.data["subannual"]
del unit_consistent_db.data["meta"]
unit_consistent_db = pyam.IamDataFrame(unit_consistent_db.data)

ValueError: reading multiple files not supported, please use `pyam.IamDataFrame.append()`

In [21]:
decomposer = mi.DecomposeCollectionTimeDepRatio(unit_consistent_db)
results = decomposer.infill_components(
    aggregate, components, 
    kyoto_df.filter(year=unit_consistent_db.filter(variable="Emissions|F-Gases")["year"].unique())
)
results.head()

,model,scenario,region,variable,unit,year,value
0,1.5C consistent,high,World,Emissions|CH4,Mt CO2-equiv/yr,2010,9.651187
1,1.5C consistent,high,World,Emissions|CH4,Mt CO2-equiv/yr,2015,10.010907
2,1.5C consistent,high,World,Emissions|CH4,Mt CO2-equiv/yr,2020,10.187857
3,1.5C consistent,high,World,Emissions|CH4,Mt CO2-equiv/yr,2025,7.503700
4,1.5C consistent,high,World,Emissions|CH4,Mt CO2-equiv/yr,2030,5.180608


In [22]:
results.variables(True)

,variable,unit
0,Emissions|CH4,Mt CO2-equiv/yr
1,Emissions|CO2,Mt CO2/yr
2,Emissions|F-Gases,Mt CO2-equiv/yr
3,Emissions|N2O,Mt CO2-equiv/yr
